In [18]:
import selenium.webdriver as webdriver
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os

# --- Configuration ---
DOWNLOAD_DIR = os.path.join(os.getcwd(), "Pexels_Downloads") 

if not os.path.exists(DOWNLOAD_DIR):
    os.makedirs(DOWNLOAD_DIR)

chrome_options = webdriver.ChromeOptions()
prefs = {
    "download.default_directory": DOWNLOAD_DIR,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True 
}
chrome_options.add_experimental_option("prefs", prefs)


# --- Initialization ---
Driver = webdriver.Chrome(options=chrome_options)
URL = "https://www.pexels.com/search/person%20selfie%20with%20tongue/"
Driver.get(URL)

DOWNLOAD_BUTTON_SELECTOR = "a[title='Download']" 
# UPDATED: Using the specific SVG class name you provided, prefixed with a dot (.) for CSS selector
CLOSE_BUTTON_SELECTOR = ".Icon_color-greyMinus2DFDFE0__D6XdE" 
actions = ActionChains(Driver)

time.sleep(5)

# --- Core Logic Variables ---
unique_thumbnails = set()
current_scroll_position = 0
SCROLL_INCREMENT = 500 


# --- Main Loop: Scroll, Find, and Download ---
print("Starting scrolling and sequential download...")
print(f"Images will be saved to: {DOWNLOAD_DIR}")

for _ in range(10):
    try:
        current_scroll_position += SCROLL_INCREMENT
        Driver.execute_script(f"window.scrollTo(0, {current_scroll_position});")
        time.sleep(4)
        
        current_thumbnails = Driver.find_elements(By.CLASS_NAME, "MediaCard_overlay__HNJfd")
        
        for thumbnail in current_thumbnails:
            if thumbnail not in unique_thumbnails:
                
                unique_thumbnails.add(thumbnail)
                
                # 1. Hover to reveal the download button
                actions.move_to_element(thumbnail).perform()
                time.sleep(0.5) 
                
                # 2. Find and click the download button
                download_button = Driver.find_element(By.CSS_SELECTOR, DOWNLOAD_BUTTON_SELECTOR)
                download_button.click()
                
                print(f"-> Download clicked for new image. Total unique downloaded: {len(unique_thumbnails)}")
                
                # 3. Wait explicitly for the close button to be clickable
                # The close button is an SVG, so we wait for its parent container to be clickable
                WebDriverWait(Driver, 10).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, CLOSE_BUTTON_SELECTOR))
                )

                # 4. Click the close icon using the confirmed CSS selector
                close_button = Driver.find_element(By.CSS_SELECTOR, CLOSE_BUTTON_SELECTOR)
                close_button.click()
                
                time.sleep(1) 
        
    except Exception:
        pass 
        
print(f"\nFinal total unique downloads initiated: {len(unique_thumbnails)}")

Driver.quit()

Starting scrolling and sequential download...
Images will be saved to: c:\Users\Nagesh Agrawal\OneDrive\Desktop\AI\Pexels_Downloads
-> Download clicked for new image. Total unique downloaded: 1

Final total unique downloads initiated: 9
